# フレア検出 学習ノート 05: 複数セクタ・複数恒星の統計と比較

このノートでは、`BaseFlareDetector` で得られるフレア検出結果をまとめて扱い、
複数セクタ・複数恒星のフレア活動を統計的に比較する方法を学びます。

ここでは、以下の 3 つの観点に注目します。

1. クラス変数に蓄積されるサマリ値の意味
2. 単一恒星内でのセクタ間比較
3. 複数恒星の間でのフレア発生率・エネルギー率の比較


## このノートで学ぶこと

- `array_flare_ratio` / `array_energy_ratio` / `array_observation_time` などの役割
- 単一恒星内でのセクタごとの FFD 的な図の作り方（簡易版）
- 複数恒星のフレア発生率やエネルギー率の比較プロットの作り方


In [1]:
import sys
from pathlib import Path

NOTEBOOK_DIR = Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("PROJECT_ROOT:", PROJECT_ROOT)

NOTEBOOK_DIR: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/notebooks/learning
PROJECT_ROOT: /Users/daisukeyamashiki/Documents/kyoto-flare-detection


## 1. クラス変数に蓄積される情報の整理

`BaseFlareDetector.process_data()` を呼ぶたびに、

- `array_flare_ratio` : フレア発生率 [1/day]
- `array_energy_ratio` : フレアエネルギーの総和 / 観測時間 [erg/day]
- `array_observation_time` : 観測の代表時刻（たとえば BJD の中央値）
- `array_starspot` : 推定されたスポット面積 [m^2]（cm^2にする場合は×1e4）
- `array_starspot_ratio` : 光度変動から見積もるスポット被覆率
- `array_data_name` : 各データの ID（FITS ファイル名から抽出）
- `array_per`, `array_per_err` : 回転周期とその不確かさ

といった値がクラス変数として蓄積されていきます。
複数の FITS ファイル（複数セクタ・複数恒星）に対して `process_data()` を
順に呼び出すことで、これらの配列を「簡易なカタログ」として扱うことができます。



In [2]:
import re
import numpy as np
import pandas as pd
import plotly.express as px

from src.base_flare_detector import BaseFlareDetector
from src.flarepy_EK_Dra import FlareDetector_EK_Dra
from src.flarepy_DS_Tuc_A import FlareDetector_DS_Tuc_A
from src.flarepy_V889_Her import FlareDetector_V889_Her

DATA_DIR_EK_DRA = PROJECT_ROOT / "data" / "TESS" / "EK_Dra"
DATA_DIR_DS_TUC_A = PROJECT_ROOT / "data" / "TESS" / "DS_Tuc_A"
DATA_DIR_V889_HER = PROJECT_ROOT / "data" / "TESS" / "V889_Her"

print("DATA_DIR_EK_DRA:", DATA_DIR_EK_DRA)
print("DATA_DIR_DS_TUC_A:", DATA_DIR_DS_TUC_A)
print("DATA_DIR_V889_HER:", DATA_DIR_V889_HER)


DATA_DIR_EK_DRA: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/EK_Dra
DATA_DIR_DS_TUC_A: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/DS_Tuc_A
DATA_DIR_V889_HER: /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/V889_Her


In [3]:
def extract_sector(data_name: str) -> int:
    """データ名から TESS セクタ番号 (sXXXX) を抽出する簡易関数。"""
    m = re.search(r"-s(\d{4})-", data_name)
    return int(m.group(1)) if m else -1


def run_detectors_for_all_stars(max_files_per_star: int = 3) -> pd.DataFrame:
    """EK Dra / DS Tuc A / V889 Her について、いくつかの FITS を処理してサマリ表を作る。"""
    rows: list[dict] = []

    star_configs = [
        ("EK Dra", DATA_DIR_EK_DRA, FlareDetector_EK_Dra),
        ("DS Tuc A", DATA_DIR_DS_TUC_A, FlareDetector_DS_Tuc_A),
        ("V889 Her", DATA_DIR_V889_HER, FlareDetector_V889_Her),
    ]

    for star_name, data_dir, Detector in star_configs:
        if not data_dir.exists():
            print(f"[warning] {star_name} のディレクトリ {data_dir} が見つかりません。")
            continue

        fits_files = sorted(data_dir.glob("*.fits"))[:max_files_per_star]
        if not fits_files:
            print(f"[warning] {star_name} の FITS ファイルが見つかりません。")
            continue

        for fpath in fits_files:
            print(f"Processing {star_name}: {fpath.name}")
            det = Detector(file=str(fpath), process_data=True)

            if det.precise_obs_time <= 0:
                flare_ratio = np.nan
                energy_ratio = np.nan
            else:
                flare_ratio = det.flare_number / det.precise_obs_time
                energy_ratio = det.sum_flare_energy / det.precise_obs_time

            rows.append(
                {
                    "star": star_name,
                    "data_name": det.data_name,
                    "sector": extract_sector(det.data_name or ""),
                    "flare_ratio": flare_ratio,
                    "energy_ratio": energy_ratio,
                    "starspot": det.starspot,
                    "starspot_ratio": det.starspot_ratio,
                    "rotation_period": det.per,
                    "rotation_period_err": det.per_err,
                }
            )

    df = pd.DataFrame(rows)
    return df


summary_df = run_detectors_for_all_stars(max_files_per_star=2)
summary_df.head()


Processing EK Dra: tess2019198215352-s0014-0000000159613900-0150-s_lc.fits
Processing EK Dra: tess2019226182529-s0015-0000000159613900-0151-s_lc.fits
Processing DS Tuc A: tess2018206045859-s0001-0000000410214986-0120-s_lc.fits
Processing DS Tuc A: tess2020186164531-s0027-0000000410214986-0189-s_lc.fits
[warning] V889 Her のディレクトリ /Users/daisukeyamashiki/Documents/kyoto-flare-detection/data/TESS/V889_Her が見つかりません。


,star,data_name,sector,flare_ratio,energy_ratio,starspot,starspot_ratio,rotation_period,rotation_period_err
0,EK Dra,tess2019198215352-s0014,-1,0.193095,3.204434e+33,8.624788e+16,0.032115,2.648040,0.109977
1,EK Dra,tess2019226182529-s0015,-1,0.200324,4.159948e+33,1.164377e+17,0.043356,2.668446,0.121133
2,DS Tuc A,tess2018206045859-s0001,-1,0.226203,4.318079e+33,3.385568e+17,0.074092,2.852273,0.159811
3,DS Tuc A,tess2020186164531-s0027,-1,0.385663,1.254789e+34,4.932592e+17,0.107948,3.633028,0.571119


上のセルでは、各恒星について最大 2 ファイルずつ処理し、

- `flare_ratio` : フレア発生率 [1/day]
- `energy_ratio` : フレアエネルギー率 [erg/day]
- `starspot` / `starspot_ratio` : スポット面積と被覆率
- `rotation_period` : 回転周期
- `sector` : TESS セクタ番号

を 1 行ずつのレコードにまとめています。
この `summary_df` をもとに、セクタ間・恒星間の比較プロットを作っていきます。


## 2. 単一恒星内のセクタ間比較（例：EK Dra）

まずは EK Dra だけを取り出し、セクタごとのフレア発生率とエネルギー率を
簡単な散布図で眺めてみます。


In [4]:
ek_df = summary_df[summary_df["star"] == "EK Dra"].copy()

if ek_df.empty:
    print("EK Dra のデータがありません。上のセルで summary_df を作成できているか確認してください。")
else:
    ek_df = ek_df.sort_values("sector")

    fig = px.scatter(
        ek_df,
        x="sector",
        y="flare_ratio",
        size="energy_ratio",
        hover_data=["data_name"],
        labels={"sector": "TESS Sector", "flare_ratio": "Flare Rate [1/day]"},
        title="EK Dra: セクタごとのフレア発生率とエネルギー率 (バブルサイズ)",
    )
    fig.update_yaxes(type="log")
    fig.show()


図では、

- x 軸: セクタ番号
- y 軸: フレア発生率（対数軸）
- バブルの大きさ: エネルギー率

として表示しています。セクタごとの観測時間や検出閾値の違いによって、
どの程度フレア活動が変動して見えるかを直感的に把握できます。


## 3. 複数恒星の比較プロット

次に、EK Dra / DS Tuc A / V889 Her を 1 枚の図にまとめ、

- 回転周期
- フレア発生率
- スポット被覆率

の関係をざっくり眺めてみます。


In [5]:
if summary_df.empty:
    print("summary_df が空です。上のセルで run_detectors_for_all_stars を実行してください。")
else:
    fig = px.scatter(
        summary_df,
        x="rotation_period",
        y="flare_ratio",
        color="star",
        size="starspot_ratio",
        hover_data=["sector", "data_name"],
        labels={
            "rotation_period": "Rotation Period [day]",
            "flare_ratio": "Flare Rate [1/day]",
            "starspot_ratio": "Starspot Filling Factor (relative)",
        },
        title="複数恒星における回転周期・フレア発生率・スポット比の関係",
    )
    fig.update_xaxes(type="log")
    fig.update_yaxes(type="log")
    fig.show()


この図から、

- 回転が速い星ほどフレア発生率が高いか？
- スポット被覆率が大きい星ほどフレアが多いか？

といった物理的な傾向を、ざっくりと可視化できます。
より厳密な解析を行う場合は、観測時間や閾値設定の違いをそろえたうえで
FFD フィッティングなどを行う必要がありますが、このノートの目的は
「クラス変数に蓄積された情報から、素早く比較プロットを作る」ことにあります。
